In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor
from math import sqrt
import env
import warnings
warnings.filterwarnings("ignore")

from scipy import stats
import acquire

In [35]:
df1 = acquire.wrangle_zillow()

In [38]:
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/zillow'
query = 'select calculatedfinishedsquarefeet as sqft, bedroomcnt as bedrooms, regionidcity as zip, \
            bathroomcnt as bathrooms, taxvaluedollarcnt as tax_value \
                from properties_2017 as props\
                    join predictions_2017 as preds using(parcelid)\
                        where props.parcelid = preds.parcelid and\
                            propertylandusetypeid = 261 and \
                                bathroomcnt <= 3 and \
                                    bedroomcnt <=4;'

In [39]:
df1 = pd.read_sql(query, url)

In [41]:
train_validate, test = train_test_split(df1, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

In [42]:
train

,sqft,bedrooms,zip,bathrooms,tax_value
19493,868.0,2.0,18098.0,2.0,161342.0
1890,1203.0,3.0,118694.0,2.0,215388.0
34995,1434.0,3.0,4406.0,2.0,78153.0
17143,1379.0,4.0,118694.0,2.0,330000.0
29266,1867.0,3.0,16764.0,2.5,565194.0
...,...,...,...,...,...
21999,1983.0,3.0,12447.0,2.0,440100.0
22008,804.0,2.0,12447.0,1.0,286063.0
15179,1312.0,2.0,20008.0,1.0,319000.0
3309,768.0,2.0,17882.0,1.0,102577.0


In [43]:
X_train_scaled = train[['sqft', 'bedrooms', 'zip', 'bathrooms']]
y_train = train['tax_value']

In [44]:
f_selector = SelectKBest(f_regression, k = 2)

f_selector.fit(X_train_scaled, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').